In [1]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk

/Users/olgacravesana/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


- how the sub-products are positioneted in the pdf for each products
- creating class for each of the main product
- doing same operation for each of the page talking about the same product
- creating function with 1st params(how they divide pdf in the first way) 2nd, 3rd ecc

**REMEMBER : some bank can have deposito a ordem in more than one page**

#### extract text (def from before)

In [11]:
def extract_clean(pdf,n_page):
    file = pdfplumber.open(pdf)
    page = file.pages[n_page]
    text = page.extract_text().lower()
    text = re.sub('isento', '0,00', text)
    text = re.sub('n/a', str(np.nan), text)
    text=text.replace('\n','')
    return text  

In [27]:
bankinter = extract_clean('bankinter_pricelists.pdf', 8)
ctt = extract_clean('ctt.pdf',4)
abanca = extract_clean('abanca1.pdf', 4)
bic = extract_clean('bic.pdf',5 )
abanca

'entrada em vigor: 20-nov-2020  1.  contas de depósito (particulares)                                                                                   índice  1.1 depósitos à ordem  comissões  acresce  outras   euros (mín/máx)  valor anual  imposto  condições produtos: fora de comercialização: conta d.o. particulares; conta ordenado; conta standard; conta future; conta kids; conta global em comercialização: conta base; conta private; conta value; conta smart; conta futuro; conta serviços mínimos bancários, conta moeda estrangeira e conta abanca internacional. as comissões abaixo indicadas ocorrem no momento em que se verifica a operação e aplicam-se às contas acima identificadas, salvo nos casos em que expressamente se exclui.  1.  emissão de extrato     1.1  mensal (enviado ao domicílio)  0,00 nan  iva – 23% 1.2  outros, para além do indicado em 1.1  2,00 por extrato 1.3  2ª via      7,50   2.   fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  iva – 23%   

In [4]:
#params 1 --> define start of table
table_bankinter = re.split('1.1\D',bankinter)[1]
table_ctt = re.split('1.1\D',ctt)[1]
table_abanca = re.split('1.1',abanca)[1:]
table_bic = re.split('1.1', bic)[1]
table_abanca = ''.join(table_abanca)

- bankinter 1st params == 1.1
- ctt 1st params == 1.1
- abanca 1st parmans == 1.1 --> len of 4, just take the first one as separatore (then join the other)
- bic 1st params == 1.1 

**in the function you have to calculate the len and if there is one more then join the rest in a list**

In [13]:
# tokens = nltk.word_tokenize(table_bankinter)
# table_ctt = ' . '.join(table_ctt.split('\n'))
tagged = nltk.sent_tokenize(table_ctt)
tagged

[' depósitos à ordem .',
 'comissões .',
 'acresce  .',
 'euros  outras condições .',
 'valor anual imposto .',
 '(mín/máx) .',
 'contas de depósito à ordem (comissões aplicáveis a todas as contas) .',
 'comissões durante a vigência do contrato .',
 '1.extrato integrado mensal 0,00 0,00 -- -- .',
 '2.consulta de saldo de conta do com comprovativo 0,50 -- is - 4% comissão por emissão ao balcão (1) .',
 '3.consulta de movimentos de conta do com .',
 '2,00 -- is - 4% comissão por emissão ao balcão .',
 'comprovativo .',
 '4.emissão 2ªs vias de avisos e outros documentos 5,00 -- iva - 23% comissão por emissão ao balcão .',
 '5.emissão 2ª via - extrato integrado 10,00 -- iva - 23% comissão por emissão ao balcão .',
 'conta banco ctt  .',
 'comissões durante a vigência do contrato .',
 '1.comissões por descoberto bancário vide secção 2.5. descobertos bancários .',
 '2.levantamento de numerário ao balcão comissão por levantamento ao  .',
 '4,50 -- is - 4% .',
 'a partir de 2 levantamentos ao 

In [26]:
table_abanca = re.sub('[0-9]{1}\.[0-9]{1,2}\D', ' . ', table_abanca)
# tables = []
# for table in table_abanca:
#     table = re.sub('[0-9]{1}\.[0-9]{1,2}\D',' . ', table_abanca)
#     tables.append(table)
# table_abanca = ' . '.join(tables)

tagged = nltk.sent_tokenize(table_abanca)
# tagged = re.sub('\n ','',tagged)
# table_abanca
lista = []
# for each in tagged:
#     each = re.sub('\n', '', table_abanca)
#     lista.append([each])
# # pd.DataFrame(lista) 
tagged
contas=[]
tagged
for tag in tagged:
    if 'conta' == tag.split()[0]:
        tag=tag.replace(' \n .','')
        contas.append(tag)
contas
# newby = []
# for conta in contas:
#     conta = conta.replace(',','.')
#     conta = conta.replace(' ',',').replace(',,',',')
#     conta = conta.split()
        
#     newby.append(conta)
# pd.DataFrame(newby)

['conta d.o.',
 'conta ordenado (nota 1 e nota 16)  0,00  nan',
 'conta standard (não admite novas contratações)  0,00  nan',
 'conta future (nota 3)  0,00  nan',
 'conta kids (nota 4)  0,00  nan \nis – 4%  ',
 'conta global (nota 5)  6,53 / 7,23 mensal  85,14',
 'conta base (nota 6)  10,00 mensal  120,00 ',
 'conta private (nota 7)   8,00 mensal  96,00',
 'conta value (nota 8)  5,00 mensal  60,00',
 'conta smart (nota 9)  5,00 mensal  60,00',
 'conta futuro (nota 10)  0,00  nan',
 'conta serviços mínimos bancários (nota 11)  2,00 semestral  4,00',
 'conta moeda estrangeira (nota 12)  5,00 mensal  60,00',
 'conta abanca internacional (nota 17)  0,00  nan \n4.   levantamento de numerário        \n  .',
 'conta para clientes dos 18 aos 28 anos.',
 'conta para clientes dos 0 aos 17 anos.',
 'conta que permite aceder aos seguintes produtos e serviços, mediante o pagamento de \numa comissão mensal fixa: constituição, manutenção, gestão e titularidade da conta de depósito à ordem; disponibil

# back to table, desperation is coming

In [ ]:
pdf = 'ctt.pdf'
file = pdfplumber.open(pdf)
page = file.pages[4]
table = page.extract_table()

In [ ]:
ctt_df = pd.DataFrame(table)

In [ ]:
ctt_df.iloc[5,1]